# sentencepiece를 활용해 vocab 만들기

## libraries needed

In [2]:
!pip install tqdm
!pip install pandas
!pip install bs4
!pip install wget
!pip install pymongo

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=02ee13809279048191f8c2ffba1a0448a29198520c239b4af774c30acbd152c7
  Stored in directory: /Users/sojeonglee/Library/Caches/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.6/645.6 kB 7.1 MB/s eta 0:00:00


In [6]:
import pandas as pd

## csv file to text

In [10]:
with open('data/news2.csv', 'r', encoding='utf-8') as file:
    first_line = file.readline()
    print(repr(first_line))  # 파일의 첫 번째 줄을 출력


',title,text,label\n'


In [11]:
in_file = "data/news2.csv"
out_file = "news.txt"
SEPARATOR = ','
df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
with open(out_file, "w") as f:
  for index, row in df.iterrows():
    f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
    f.write("\n\n\n\n") # 구분자

### install google sentence piece 설치하기

In [12]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00


## vocab 만들기

In [14]:
import sentencepiece as spm

corpus = "news.txt"
prefix = "news2"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=news.txt --model_prefix=news2 --vocab_size=8007 --model_type=bpe --max_sentence_length=999999 --pad_id=0 --pad_piece=[PAD] --unk_id=1 --unk_piece=[UNK] --bos_id=2 --bos_piece=[BOS] --eos_id=3 --eos_piece=[EOS] --user_defined_symbols=[SEP],[CLS],[MASK]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: news.txt
  input_format: 
  model_prefix: news2
  model_type: BPE
  vocab_size: 8007
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: [SEP]
  user_defined_symbo

 piece=▁Nor
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=254 size=5240 all=90672 active=4802 piece=▁sales
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=253 size=5260 all=90813 active=4943 piece=▁wealthy
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=251 size=5280 all=90973 active=5103 piece=anges
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=250 size=5300 all=91069 active=5199 piece=▁affili
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=250 min_freq=101
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=249 size=5320 all=91187 active=4666 piece=▁insisted
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=247 size=5340 all=91420 active=4899 piece=osen
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=246 size=5360 all=91566 active=5045 piece=▁lobby
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=245 size=5380 all=91687 active=5166 piece=▁realize
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=244 size=5400 all=91817 active=5296 piece=▁machines
bpe_model_trainer.cc

# test model

In [15]:
import sentencepiece as spm

vocab_file = "news2.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

겨울이 되어서 날씨가 무척 추워요.
['▁', '겨울이', '▁', '되어서', '▁', '날씨가', '▁', '무척', '▁', '추워요', '.']
[7921, 1, 7921, 1, 7921, 1, 7921, 1, 7921, 1, 7942]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁', '이번', '▁', '성탄절은', '▁', '화이트', '▁', '크리스마스가', '▁', '될까요', '?']
[7921, 1, 7921, 1, 7921, 1, 7921, 1, 7921, 1, 7990]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁', '겨울에', '▁', '감기', '▁', '조심하시고', '▁', '행복한', '▁', '연말', '▁', '되세요', '.']
[7921, 1, 7921, 1, 7921, 1, 7921, 1, 7921, 1, 7921, 1, 7942]

